In [6]:
import pandas as pd
import getpass
from sqlalchemy import create_engine
from sqlalchemy.exc import ResourceClosedError

In [7]:
class MySQLConnection:
    def __init__(self,user,hostname,db):
        self.user = str(user)
        self.hostname = str(hostname)
        self.db = str(db)
        pword = getpass.getpass("Enter password for user {}".format(user))
        # self.cnx = mysql.connector.connect(host=self.hostname,user=self.user, password=pword,database=self.db)
        self.engine = create_engine("mysql+mysqlconnector://{}:{}@{}/{}".format(user,pword,hostname,db))
        self.cnx = self.engine.connect()
    
    def write_to_db(self, df, table_name):
        try:
            df.to_sql(table_name,con = self.cnx,if_exists='fail',index=False)
        except Exception as e:
            print("\n SQL Write error with: ")
            print(df,"\n", e)
    
    def query(self,q):
        try:
            res = pd.read_sql(q,self.cnx)
            return res
        except ResourceClosedError:
            print("Query success, nothing was returned")

In [8]:
sql = MySQLConnection('oskars_s','ec2-13-53-140-213.eu-north-1.compute.amazonaws.com','dati')

Enter password for user oskars_s········


In [8]:
select_query_template = """ 
SELECT 
    Username, 
    Identifier 
FROM 
    username_or_email 
WHERE 
    identifier > 4000
ORDER BY 
    identifier DESC 
LIMIT 3
"""

pd.read_sql(select_query_template, sql.cnx)

,Username,Identifier
0,jenkins46,9346
1,booker12,9012
2,smith79,5079


In [9]:
grouping_query_template = """
select
    Location,
    count(*) 
from 
    username_password_recovery_code 
group by 
    Location
"""
# username_password_recovery_code
pd.read_sql(grouping_query_template, sql.cnx)

,Location,count(*)
0,London,2
1,Manchester,3


In [10]:
join_query_template = """
select
    username.username,
    email.Identifier
from 
    username inner join email on 
    username.Identifier = email.Identifier
"""

pd.read_sql(join_query_template, sql.cnx)

,username,Identifier
0,grey07,2070
1,johnson81,4081
2,jenkins46,9346
3,smith79,5079


# CREATE TABLE  
Šie divi atslēgvārdi izveido tabulu ar fiksētu kolonnu skaitu un fiksētiem kolonnu vērtību tipiem.  
Piemērs:  
CREATE TABLE Tabula1 (Kolonna1 Int, Kolonna2 Int, Kolonna3 char)  

In [ ]:
izveidosanas_vaicajums = """
create table tabula1(kolonna1 int , kolonna2 int, kolonna3 char)
"""

# pd.read_sql(izveidosanas_vaicajums, sql.cnx)

# SELECT atslēgvārds  
SELECT atslēgvārds atlasa kolonnas. Select atslēgvārdam ir jābūt jebkurā SQL vaicājumā. Select kā argumentus ņem vai nu attiecīgās tabulas kolonnas vai arī īpašu argumentu * , kas atlasa visas kolonnas

In [ ]:
select_vaicajums = """
select * from email
"""
pd.read_sql(select_vaicajums, sql.cnx)

# FROM atslēgvārds  
Tāpat kā SELECT, arī FROM atslēgvārds ir atrodams jebkurā SQL vaicājumā. Loģika aiz šī ir tāda, ka mēs vienmēr ņemam kādas kolonnas no kādas tabulas.  
**Visas kolonnas vienmēr tiek ņemtas no kādas tabulas, neatkarīgi no tā vai tabula ir jau esoša vai tiek izveidota kāda cita vaicājuma rezultātā.**  

# WHERE atslēgvārds  
**WHERE atslēgvārds NAV obligāts. WHERE atslēgvārds tiek izmantots, lai atlasītu rindas, kuras atbilst 'būla mainīgā' apgalvojumam piem:**  

SELECT  
  Kolonna1,  
  Kolonna2  
FROM Tabula1  
WHERE Kolonna1 > 10  

**WHERE atslēgvārdam var padot vairākus argumentus, kurus var apvienot ar AND piem:**

SELECT  
  Kolonna1,  
  Kolonna2  
FROM Tabula1  
WHERE Kolonna1 > 10  AND Kolonna2 < 4  



In [ ]:
a = 4
b = 5

d = 6
e = 7

f = d == e
c = a == b 

f and c

# ORDER BY funkcija/atslēgvārdi  
Order by funkcija SQL vaicājumā vienmēr būs pēc WHERE atslēgvārda, bet pirms LIMIT, ja tāds tiek lietots. Sakārto tabulu vai nu augoši vai dilstoši. Piemērs:  
SELECT * FROM Tabula1 ORDER BY <kolonnas_nosaukums> <asc/desc>

In [ ]:
order_vaicajums = """
select * from email order by identifier desc
"""

pd.read_sql(order_vaicajums, sql.cnx)

# Tabulas var veidot arī ar SELECT atslēgvārdu  
**Izmantojot CREATE TABLE, tipiski ir jānorāda tabulas nosaukums, kolonnu nosaukumi un to tipi, bet izveidot tabulu var arī ar sekojošo sintaksi:**  

CREATE TABLE skelums AS SELECT Kolonna1, Kolonna2  
FROM Tabula1  

**Šis strādā, jo, ja mēs atsaucamies uz eksistējošu tabulu, tās kolonnām jau ir gan nosaukumi, gan tipi un tie nav jānorāda vēlreiz**  


In [ ]:
# Uzdevums:
# 1. Ar select <kolonnas> from <tabula>' izpētiet tabulas 'email' un 'username', 
# 2. Izveidojiet tabulu username_sample no tabulas username, kurā ir tikai kolonnas 'Identifier' un 'username' tikai 
# no tiem darbiniekiem, kuriem Identificator ir lielāks par 4000

In [ ]:
# uzdevums = """
# create table username_sample as select Identifier, username from username where Identifier > 4000
# """

uzdevums = """
select 
    * 
from 
    username_sample
"""

pd.read_sql(uzdevums,sql.cnx)

# Agregācijas funkcijas: COUNT, SUM, MIN, MAX, AVG  

In [ ]:
agr_skaita_vaicajums = """
select count(distinct(firstname)) from email where identifier > 4000
"""
pd.read_sql(agr_skaita_vaicajums,sql.cnx)

In [ ]:
agr_sum_vaicajums = """
select sum(*) from email
"""

# agr_sum_vaicajums = """
# select sum(loginemail) from email
# """

agr_sum_vaicajums = """
select sum(Identifier) from email
"""

# agr_sum_vaicajums = """
# select count from email
# """

pd.read_sql(agr_sum_vaicajums,sql.cnx)

In [ ]:
agr_sum_vaicajums = """
select max(Identifier) from email
"""

pd.read_sql(agr_sum_vaicajums,sql.cnx)

In [ ]:
agr_sum_vaicajums = """
select count(*) from email
"""
pd.read_sql(agr_sum_vaicajums,sql.cnx)

# JOIN atslēgvārds  
Viena no spēcīgākajām SQL iespējām ir salīdzinoši vienkāršā tabulu sapludināšana.  
JOIN atslēgvārds izveido jaunu tabulu no divām citām tabulām, apvienojot rindiņas.  
Rindiņas tiek apvienotas izmantojot kādas norādītas kolonnas vērtības.  
**Svarīgi: JOIN atslēgvārds tabulas izveido, bet lai kaut kas tiktu atgriezts, vaicājumam tāpat ir jāsatur SELECT.**  
Piemērs:  
SELECT < kolonnas > FROM <join apgalvojums šeit>

Ir 4 apvienošanas stratēģijas 'inner', 'outer', 'left' un 'right'

In [10]:
# # Uzdevums:
# # Sekojošajā piemērā izstājiet atslēgvārdu 'inner' ar secīgi 'left' un 'right'. Ko var secināt?

# join_query_template = """
# select
#     username_sample.username,
#     email.Identifier
# from 
#     username_sample inner join email on 
#     username_sample.Identifier = email.Identifier
# """

# pd.read_sql(join_query_template, sql.cnx)
sql.query("select * from username")

,Username,Identifier,Firstname,Lastname
0,booker12,9012,Rachel,Booker
1,grey07,2070,Laura,Grey
2,johnson81,4081,Craig,Johnson
3,jenkins46,9346,Mary,Jenkins
4,smith79,5079,Jamie,Smith


In [12]:
sql.query("select * from email")

,Loginemail,Identifier,Firstname,Lastname
0,laura@example.com,2070,Laura,Grey
1,craig@example.com,4081,Craig,Johnson
2,mary@example.com,9346,Mary,Jenkins
3,jamie@example.com,5079,Jamie,Smith


In [19]:
basic_join = """
select 
    username.Firstname,
    Loginemail,
    Username
from 
    username left join email on 
    username.Identifier=email.Identifier
"""
sql.query(basic_join)

,Firstname,Loginemail,Username
0,Laura,laura@example.com,grey07
1,Craig,craig@example.com,johnson81
2,Mary,mary@example.com,jenkins46
3,Jamie,jamie@example.com,smith79
4,Rachel,None,booker12


![alt text](http://codinghelptech.com/blog_post/Sql-Join.png "Logo Title Text 1")
<!-- http://codinghelptech.com/Database/Sql-Join.html -->

In [ ]:
# Uzdevums / MAJAS DARBS:
# Izveidojiet tabulu, kurā ir TIKAI visu lietotāju lietotājvārdi e-pasti un atrašanās vietas,
# izmantojot tikai tabulas, kuras iegūtas no uzskaititajiem failiem. Mums interese tikai ieraksti, kur nekas netrukst.
# 1. email.csv
# 2. username-password-recovery-code.csv

In [22]:
sql.query("select * from email")

,Loginemail,Identifier,Firstname,Lastname
0,laura@example.com,2070,Laura,Grey
1,craig@example.com,4081,Craig,Johnson
2,mary@example.com,9346,Mary,Jenkins
3,jamie@example.com,5079,Jamie,Smith


In [23]:
sql.query("select * from username_password_recovery_code")

,Username,Identifier,Onetimepassword,Recoverycode,Firstname,Lastname,Department,Location
0,booker12,9012,12se74,rb9012,Rachel,Booker,Sales,Manchester
1,grey07,2070,04ap67,lg2070,Laura,Grey,Depot,London
2,johnson81,4081,30no86,cj4081,Craig,Johnson,Depot,London
3,jenkins46,9346,14ju73,mj9346,Mary,Jenkins,Engineering,Manchester
4,smith79,5079,09ja61,js5079,Jamie,Smith,Engineering,Manchester


In [26]:
sql.query("select * from username")

,Username,Identifier,Firstname,Lastname
0,booker12,9012,Rachel,Booker
1,grey07,2070,Laura,Grey
2,johnson81,4081,Craig,Johnson
3,jenkins46,9346,Mary,Jenkins
4,smith79,5079,Jamie,Smith


In [24]:
basic_join = """
SELECT
    username.Firstname,
    Loginemail,
    Username
FROM 
    username JOIN email ON 
    username.Identifier=email.Identifier
"""



vaic = """
SELECT
    loginemail,
    location
FROM 
    username_password_recovery_code INNER JOIN email ON 
    username_password_recovery_code.Identifier = email.Identifier
"""



In [25]:
sql.query(vaic)

,loginemail,location
0,laura@example.com,London
1,craig@example.com,London
2,mary@example.com,Manchester
3,jamie@example.com,Manchester


# GROUP BY atslēgvārds  
<!-- https://www.w3schools.com/sql/sql_groupby.asp-->
GROUP BY atslēgvārds ir komanda, kas ir nevis paredzēts lai filtrētu vai atgrieztu tabulas, bet gan lai sagrupētu tabulas pēc kādas kolonnas unikālajām vērtībām.